##### import libraries

In [66]:
import pandas as pd
import numpy as np

In [67]:
import pandas as pd
import numpy as np
import numbers

from IPython.core.display_functions import display


def normalize_data(df: pd.DataFrame, test_date: pd.DataFrame = None, ignore_cols: list = []):
    ignore_cols.append('class')
    ignore_cols.append('id')
    numeric_cols = df.select_dtypes(include='number').columns
    numeric_cols = [col for col in numeric_cols if col not in ignore_cols]

    for col in numeric_cols:
        vals = df[col]
        mn = min(vals)
        mx = max(vals)
        if 0 <= mn and mx <= 1:
            continue
        denom = mx - mn

        df[col] = df.apply(lambda row: (row[col] - mn) / denom, axis=1)
        if test_date is not None:
            test_date[col] = test_date.apply(lambda row: (row[col] - mn) / denom, axis=1)

    non_numeric_cols = df.select_dtypes(include='object').columns.tolist()
    non_numeric_cols = [col for col in non_numeric_cols if col not in ignore_cols]

    for col in non_numeric_cols:
        uniq_vals = df[col]
        uniq_vals = uniq_vals.unique()
        print(f'normilize order of "{col}" :: {uniq_vals}')
        vals_map = {}
        for val in range(len(uniq_vals)):
            vals_map[uniq_vals[val]] = [0] * len(uniq_vals)
            vals_map[uniq_vals[val]][val] = 1

        df[col] = df.apply(lambda row: vals_map[row[col]], axis=1)
        if test_date is not None:
            test_date[col] = test_date.apply(lambda row: vals_map[row[col]], axis=1)


def jaccard_coefficient(l1, l2):
    # a is the number combinations of 1 and 0
    a = len([1 for i, j in zip(l1, l2) if i == 1 and j == 0])
    # b is the number combinations of 0 and 1
    b = len([1 for i, j in zip(l1, l2) if i == 0 and j == 1])
    # c is the number combinations of 1 and 1
    c = len([1 for i, j in zip(l1, l2) if i == j == 1])

    return (a + b) / (a + b + c)


def appariement_coefficient(l1, l2):
    # a is the number combinations of 1 and 0
    a = len([1 for i, j in zip(l1, l2) if i == 1 and j == 0])
    # b is the number combinations of 0 and 1
    b = len([1 for i, j in zip(l1, l2) if i == 0 and j == 1])
    # c is the number combinations of 1 and 1
    c = len([1 for i, j in zip(l1, l2) if i == j == 1])
    # d is the number combinations of 0 and 0
    d = len([1 for i, j in zip(l1, l2) if i == j == 0])

    return (a + b) / (a + b + c + d)


def distance(row_a: pd.Series, row_b: pd.Series, columns: list, options: dict = {}) -> float:
    """
        Calculates the distance between two rows
        :param row: the row to calculate the distance from
        :param test_data: the row to calculate the distance to
        :param ignore_columns: columns to ignore in the calculation
        :param options: options for the distance calculation
        :               options['class'] = the class column name
        :               options['col'] = the distance calculation method for the column
        :               options['col'] = 'jaccard' for jaccard coefficient
        :
        :                   1  0
        :               1   a  b
        :               0   c  d
        :           symitrique attribut :     distance_jaccard = (b + c) / (a + b + c)
        :       non symetrique attribut : distance_appariement = (b + c) / (a + b + c + d)  **** (default)
        :
        :return: the distance between the two rows
    """
    # calculate distance between row and test_data
    rest_dist = {}
    for col in columns:
        if col == 'class' or col == 'id':
            continue
        if type(row_a[col]) == list:
            if col in options and options[col] == 'jaccard':
                rest_dist[col] = jaccard_coefficient(row_a[col], row_b[col])
            else:
                rest_dist[col] = appariement_coefficient(row_a[col], row_b[col])
        elif isinstance(row_a[col], numbers.Number):
            rest_dist[col] = abs(row_a[col] - row_b[col])
        else:
            print(f'error :: {col} is not a number or a list : ', type(row_a[col]), row_a[col], row_b[col])

    dist = sum(rest_dist.values()) / len(rest_dist)
    print(f'distance of each column ::  {rest_dist} / {len(rest_dist)} -- result = {dist}')
    return dist


# def calcul_distances(df: pd.DataFrame, labels: list):
#     # calculate distance
#     Z = np.zeros((len(labels), len(labels)))
#     for i in range(len(df)):
#         for j in range(i + 1, len(df)):
#             Z[i, j] = distance(df.iloc[i], df.iloc[j], columns=df.columns)
#             Z[j, i] = Z[i, j]
#     return Z


def calcul_distances(df: pd.DataFrame, labels: list):
    # calculate distance

    ignore_columns = []
    ignore_columns.append('id')
    ignore_columns.append('class')
    jaccard = True
    for col in df.columns:
        if col in ignore_columns:
            continue
        if not df[col].isin([0, 1]).all():
            jaccard = False
            break

    Z = np.zeros((len(labels), len(labels)))
    for i in range(len(df)):
        for j in range(i + 1, len(df)):
            if not jaccard:
                Z[i, j] = distance(df.iloc[i], df.iloc[j], columns=df.columns)
            else:
                Z[i, j] = jaccard_coefficient(df.iloc[i], df.iloc[j])
            Z[j, i] = Z[i, j]
    return Z


def display_dist_matrix(labels: list, matrix: np.ndarray):
    # create a pandas dataframe from the numpy matrix with row and column labels
    df = pd.DataFrame(matrix, index=labels, columns=labels)

    # display the dataframe with labels for rows and columns
    styled_df = df.style.set_caption('My Matrix').set_table_styles(
        [{'selector': 'th', 'props': [('font-size', '14px')]}]).set_properties(
        **{'text-align': 'center', 'font-size': '12px'}).set_table_attributes('border="1"')

    display(styled_df)


# Hierarchical clustering algorithm implementation AGNES

##### Get the index of the minimum value of the matrix

In [68]:
def np_min_ign_diagonal(z: np.ndarray):
    row_index , col_index = 0, 1
    for i in range(len(z)):
        for j in range(len(z)):
            if i == j:
                continue
            if z[i, j] < z[row_index, col_index]:
                row_index = i
                col_index = j
    return row_index, col_index

# AGNES algorithm

In [69]:
def hierarchical_clustering_algo_agnes(df: pd.DataFrame, options: dict={}):
    # normalize data
    normalize_data(df, None)

    print(f'df :')
    print(df, end='\n\n')
    # l = dict['label']
    labels = list(map(str, df['id'].values))
    if 'class' in df.columns:
        df = df.drop(columns=['id', 'class'])
    else:
        df = df.drop(columns=['id'])
    Z= calcul_distances(df, labels)

    while len(Z) > 1:
        display_dist_matrix(labels, Z)
        # get the index of the minimum value of the matrix
        row_index, col_index = np_min_ign_diagonal(Z)

        # update the matrix
        for j in range(len(Z)):
            if j == col_index:
                Z[row_index, j] = 0
            elif j != row_index:
                Z[row_index, j] = min(Z[row_index, j], Z[col_index, j])
                Z[j, row_index] = Z[row_index, j]
        Z = np.delete(Z, col_index, 0)
        Z = np.delete(Z, col_index, 1)

        labels[row_index] = f'{labels[row_index]}-{labels[col_index]}'
        labels = list(np.delete(labels, col_index, 0))

    print(df)


# Test

In [70]:
data = pd.read_csv('data.csv')
labels = data['id'].values

hierarchical_clustering_algo_agnes(data)

normilize order of "cheveux" :: ['blond' 'brun' 'rousse']
df :
      id     poids    cheveux  taile  vegetarien  class
0  Sarah  0.111111  [1, 0, 0]    0.5           0      1
1   Dana  0.444444  [1, 0, 0]    1.0           1      0
2   Alex  0.333333  [0, 1, 0]    0.0           1      0
3  Annie  0.555556  [1, 0, 0]    0.0           0      1
4  Emily  0.777778  [0, 0, 1]    0.5           0      1
5    Ali  0.888889  [0, 1, 0]    1.0           0      0
6   John  1.000000  [0, 1, 0]    0.5           0      0
7  Katie  0.000000  [1, 0, 0]    0.0           1      0

distance of each column ::  {'poids': 0.3333333333333333, 'cheveux': 0.0, 'taile': 0.5, 'vegetarien': 1} / 4 -- result = 0.4583333333333333
distance of each column ::  {'poids': 0.2222222222222222, 'cheveux': 0.6666666666666666, 'taile': 0.5, 'vegetarien': 1} / 4 -- result = 0.5972222222222222
distance of each column ::  {'poids': 0.4444444444444445, 'cheveux': 0.0, 'taile': 0.5, 'vegetarien': 0} / 4 -- result = 0.23611111111111

,Sarah,Dana,Alex,Annie,Emily,Ali,John,Katie
Sarah,0.000000,0.458333,0.597222,0.236111,0.333333,0.486111,0.388889,0.402778
Dana,0.458333,0.000000,0.444444,0.527778,0.625000,0.527778,0.680556,0.361111
Alex,0.597222,0.444444,0.000000,0.472222,0.652778,0.638889,0.541667,0.250000
Annie,0.236111,0.527778,0.472222,0.000000,0.347222,0.500000,0.402778,0.388889
Emily,0.333333,0.625000,0.652778,0.347222,0.000000,0.319444,0.222222,0.736111
Ali,0.486111,0.527778,0.638889,0.500000,0.319444,0.000000,0.152778,0.888889
John,0.388889,0.680556,0.541667,0.402778,0.222222,0.152778,0.000000,0.791667
Katie,0.402778,0.361111,0.250000,0.388889,0.736111,0.888889,0.791667,0.000000


,Sarah,Dana,Alex,Annie,Emily,Ali-John,Katie
Sarah,0.000000,0.458333,0.597222,0.236111,0.333333,0.388889,0.402778
Dana,0.458333,0.000000,0.444444,0.527778,0.625000,0.527778,0.361111
Alex,0.597222,0.444444,0.000000,0.472222,0.652778,0.541667,0.250000
Annie,0.236111,0.527778,0.472222,0.000000,0.347222,0.402778,0.388889
Emily,0.333333,0.625000,0.652778,0.347222,0.000000,0.222222,0.736111
Ali-John,0.388889,0.527778,0.541667,0.402778,0.222222,0.000000,0.791667
Katie,0.402778,0.361111,0.250000,0.388889,0.736111,0.791667,0.000000


,Sarah,Dana,Alex,Annie,Emily-Ali-John,Katie
Sarah,0.000000,0.458333,0.597222,0.236111,0.333333,0.402778
Dana,0.458333,0.000000,0.444444,0.527778,0.527778,0.361111
Alex,0.597222,0.444444,0.000000,0.472222,0.541667,0.250000
Annie,0.236111,0.527778,0.472222,0.000000,0.347222,0.388889
Emily-Ali-John,0.333333,0.527778,0.541667,0.347222,0.000000,0.736111
Katie,0.402778,0.361111,0.250000,0.388889,0.736111,0.000000


,Sarah-Annie,Dana,Alex,Emily-Ali-John,Katie
Sarah-Annie,0.000000,0.458333,0.472222,0.333333,0.388889
Dana,0.458333,0.000000,0.444444,0.527778,0.361111
Alex,0.472222,0.444444,0.000000,0.541667,0.250000
Emily-Ali-John,0.333333,0.527778,0.541667,0.000000,0.736111
Katie,0.388889,0.361111,0.250000,0.736111,0.000000


,Sarah-Annie,Dana,Alex-Katie,Emily-Ali-John
Sarah-Annie,0.000000,0.458333,0.388889,0.333333
Dana,0.458333,0.000000,0.361111,0.527778
Alex-Katie,0.388889,0.361111,0.000000,0.541667
Emily-Ali-John,0.333333,0.527778,0.541667,0.000000


,Sarah-Annie-Emily-Ali-John,Dana,Alex-Katie
Sarah-Annie-Emily-Ali-John,0.000000,0.458333,0.388889
Dana,0.458333,0.000000,0.361111
Alex-Katie,0.388889,0.361111,0.000000


,Sarah-Annie-Emily-Ali-John,Dana-Alex-Katie
Sarah-Annie-Emily-Ali-John,0.000000,0.388889
Dana-Alex-Katie,0.388889,0.000000


      poids    cheveux  taile  vegetarien
0  0.111111  [1, 0, 0]    0.5           0
1  0.444444  [1, 0, 0]    1.0           1
2  0.333333  [0, 1, 0]    0.0           1
3  0.555556  [1, 0, 0]    0.0           0
4  0.777778  [0, 0, 1]    0.5           0
5  0.888889  [0, 1, 0]    1.0           0
6  1.000000  [0, 1, 0]    0.5           0
7  0.000000  [1, 0, 0]    0.0           1
